In [ ]:
param = {
  "model": 'gcit', # ['gcit']
  "sample_size": 2000, # number of samples
  "batch_size": 64,
  "z_dim": 250, # [50,100,150,200,250]
  "dx": 1,
  "dy": 1,
  "test": 'power', # ['type1error', 'power']
  "n_test": 500, ###original 500
  "n_iters": 3000, #original 1000
  "eps_std": 0.5,
  "dist_z": 'gaussian',#### choices=['gaussian', 'laplace']
  "alpha_x": 0.15, ## only used under alternative [0.15, 0.30, 0.45, 0.60, 0.75]
  "m_value": 100,
  "k_value": 2,
  "b_value": 30, #original 30
  "j_value": 1000, #original 1000##bootstrap number
  "alpha": 0.1,
  "alpha1": 0.05,
  "set_seed": 42
}

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Utilites related to Sinkhorn computations and training for TensorFlow 2.0
import tensorflow as tf
import logging
import tensorflow_probability as tfp
from sklearn.metrics.pairwise import rbf_kernel
from scipy.stats import rankdata, ks_2samp, wilcoxon
from sklearn.model_selection import KFold
from datetime import datetime
import decimal
import gc # Garbage Collector
from tqdm import tqdm



logging.getLogger('tensorflow').disabled = True
tf.keras.backend.set_floatx('float32')


'''
This code reproduces the real data experiments using the CCLE data.
Preprocessing steps follow the code of W. Tansey at https://github.com/tansey/hrt.
And the following code is obtained from https://github.com/alexisbellot/GCIT.
'''


def load_ccle(drug_target='PLX4720', feature_type='both', normalize=False):
    '''
    :param drug target: specific drug we w ant to analyse
    :param normalize: normalize data
    :return: genetic features (mutations) as a 2d array for each cancer cell and corresponding drug response measured with Amax
    '''
    if feature_type in ['expression', 'both']:
        # Load gene expression
        expression = pd.read_csv('./ccle_data/expression.txt', delimiter='\t', header=2, index_col=1).iloc[:, 1:]
        expression.columns = [c.split(' (ACH')[0] for c in expression.columns]
        features = expression
    if feature_type in ['mutation', 'both']:
        # Load gene mutation
        mutations = pd.read_csv('./ccle_data/mutation.txt', delimiter='\t', header=2, index_col=1).iloc[:,1:]
        mutations = mutations.iloc[[c.endswith('_MUT') for c in mutations.index]]
        features = mutations
    if feature_type == 'both':
        # get cells having both expression and mutation data
        both_cells = set(expression.columns) & set(mutations.columns)
        z = {}
        for c in both_cells:
            exp = expression[c].values
            if len(exp.shape) > 1:
                exp = exp[:, 0]
            z[c] = np.concatenate([exp, mutations[c].values])
        both_df = pd.DataFrame(z, index=[c for c in expression.index] + [c for c in mutations.index])
        features = both_df

    print('Genetic features dimension = {} on {} cancer cells'.format(features.shape[0], features.shape[1]))

    # Get per-drug X and y regression targets
    response = pd.read_csv('./ccle_data/response.csv', header=0, index_col=[0, 2])

    # names of cell lines, there are 504
    cells = response.index.levels[0]
    # names of drugs, there are 24
    drugs = response.index.levels[1]

    X_drugs = [[] for _ in drugs]
    y_drugs = [[] for _ in drugs]

    # subset data to include only cells, mutations and response associated with chosen drug
    for j, drug in enumerate(drugs):
            if drug_target is not None and drug != drug_target:
                continue # return to beginning of the loop
            for i, cell in enumerate(cells):
                if cell not in features.columns or (cell, drug) not in response.index:
                    continue
                # all j empty except index that corresponds to target drug
                # for this j we iteratively append all the mutations on every cell
                X_drugs[j].append(features[cell].values) # store genetic features (mutations and expression) that appear in cells
                y_drugs[j].append(response.loc[(cell, drug), 'Amax']) # store response of the drug
            print('{}: Cell number = {}'.format(drug, len(y_drugs[j])))

    # convert to np array
    X_drugs = [np.array(x_i) for x_i in X_drugs]
    y_drugs = [np.array(y_i) for y_i in y_drugs]

    if normalize:
        X_drugs = [(x_i if (len(x_i) == 0) else (x_i - x_i.min(axis=0, keepdims=True)) /
                                                (x_i.max(axis=0, keepdims=True) - x_i.min(axis=0, keepdims=True)))
                   for x_i in X_drugs]
        y_drugs = [(y_i if (len(y_i) == 0 or y_i.std() == 0) else (y_i - y_i.min(axis=0, keepdims=True)) /
                                                (y_i.max(axis=0, keepdims=True) - y_i.min(axis=0, keepdims=True)))
                   for y_i in y_drugs]

    '''
    if normalize:
        X_drugs = [(x_i if (len(x_i) == 0) else (x_i - x_i.mean(axis=0, keepdims=True)) /
        x_i.std(axis=0).clip(1e-6)) for x_i in X_drugs]
        y_drugs = [(y_i if (len(y_i) == 0 or y_i.std() == 0) else (y_i - y_i.mean()) / y_i.std()) for y_i in y_drugs]
    '''
    drug_idx = drugs.get_loc(drug_target)
    # 2d array for features and 1d array for response
    X_drug, y_drug = X_drugs[drug_idx], y_drugs[drug_idx]

    return X_drug, y_drug, features


# X_drug, y_drug, features = load_ccle(feature_type='mutation')


def ccle_feature_filter(X, y, threshold=0.1):
    '''
    :param X: features
    :param y: response
    :param threshold: correlation threshold
    :return: logical array with False for all features that do not have at least pearson correlation at threshold with y
    and correlations for all variables
    '''
    corrs = np.array([np.abs(np.corrcoef(x, y)[0, 1]) if x.std() > 0 else 0 for x in X.T])
    selected = corrs >= threshold # True/False
    print(selected.sum(), selected.shape, corrs)
    return selected, corrs

# ccle_selected, corrs = ccle_feature_filter(X_drug, y_drug, threshold=0.1)

# features.index[ccle_selected]
# stats.describe(corrs[ccle_selected])


def fit_elastic_net_ccle(X, y, nfolds=3):
    '''
    :param X: features
    :param y: response
    :param nfolds: number of folds for hyperparameter tuning
    :return: fitted elastic net model
    '''
    from sklearn.linear_model import ElasticNetCV
    # The parameter l1_ratio corresponds to alpha in the glmnet R package
    # while alpha corresponds to the lambda parameter in glmnet
    # enet = ElasticNetCV(l1_ratio=np.linspace(0.2, 1.0, 10),
    #                     alphas=np.exp(np.linspace(-6, 5, 250)),
    #                     cv=nfolds)
    enet = ElasticNetCV(l1_ratio=0.2, # It always chooses l1_ratio=0.2
                        alphas=np.exp(np.linspace(-6, 5, 250)),
                        cv=nfolds)
    print('Fitting via CV')
    enet.fit(X, y)
    alpha, l1_ratio = enet.alpha_, enet.l1_ratio_
    print('Chose values: alpha={}, l1_ratio={}'.format(alpha, l1_ratio))
    return enet

# elastic_model = fit_elastic_net_ccle(X_drug[:,ccle_selected], y_drug)


def fit_random_forest_ccle(X, y):
    '''
    :param X: features
    :param y: response
    :param nfolds: number of folds for hyperparameter tuning
    :return: fitted elastic net model
    '''
    from sklearn.ensemble import RandomForestRegressor

    rf = RandomForestRegressor()

    rf.fit(X,y)

    return rf

# rf_model = fit_random_forest_ccle(X_drug[:,ccle_selected], y_drug)


def plot_ccle_predictions(model, X, y):
    from sklearn.metrics import r2_score
    plt.close()
    y_hat = model.predict(X)
    plt.scatter(y_hat, y, color='blue')
    plt.plot([min(y.min(), y_hat.min()), max(y.max(), y_hat.max())],
             [min(y.min(), y_hat.min()),max(y.max(), y_hat.max())], color='red', lw=3)
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(' ($r^2$={:.4f})'.format( r2_score(y, y_hat)))
    plt.tight_layout()

# plot_ccle_predictions(elastic_model,X_drug[:,ccle_selected],y_drug)


def print_top_features(model):
    # model_weights = np.mean([m.coef_ for m in model.models], axis=0)
    if model == rf_model:
        model_weights = model.feature_importances_
    else:
        model_weights = model.coef_

    ccle_features = features[ccle_selected]

    print('Top by fit:')
    for idx, top in enumerate(np.argsort(np.abs(model_weights))[::-1]):
        print('{}. {}: {:.4f}'.format(idx+1, ccle_features.index[top], model_weights[top]))

# print_top_features(rf_model)
# print_top_features(elastic_model)


def run_test_ccle(X, Y):
    pval = []
    for x_index in range(X.shape[1]):
        z = np.delete(X, x_index, axis=1)
        x = X[:, x_index]
        x = x.reshape((len(x), 1))
        Y = Y.reshape((len(Y), 1))
        # now run test
        pval.append(GCIT(x, Y, z))

    ccle_features = features[ccle_selected]

    print('Top by fit:')
    for idx, top in enumerate(np.argsort(np.abs(pval))):
        print('{}. {}: {:.4f}'.format(idx+1, ccle_features.index[top], pval[top]))

# run_test_ccle(X_drug[:,ccle_selected],y_drug)



def cost_xy(x, y, scaling_coef):
    '''
    L2 distance between vectors, using expanding and hence is more memory intensive
    :param x: x is tensor of shape [batch_size, x dims]
    :param y: y is tensor of shape [batch_size, y dims]
    :param scaling_coef: a scaling coefficient for distance between x and y
    :return: cost matrix: a matrix of size [batch_size, batch_size] where
    '''
    x = tf.expand_dims(x, 1)
    y = tf.expand_dims(y, 0)
    return tf.reduce_sum((x - y)**2, -1) * scaling_coef


def benchmark_sinkhorn(x, y, scaling_coef, epsilon=1.0, L=10):
    '''
    :param x: a tensor of shape [batch_size, sequence length]
    :param y: a tensor of shape [batch_size, sequence length]
    :param scaling_coef: a scaling coefficient for squared distance between x and y
    :param epsilon: (float) entropic regularity constant
    :param L: (int) number of iterations
    :return: V: (float) value of regularized optimal transport
    '''
    n_data = x.shape[0]
    # Note that batch size of x can be different from batch size of y
    m = 1.0 / tf.cast(n_data, tf.float64) * tf.ones(n_data, dtype=tf.float64)
    n = 1.0 / tf.cast(n_data, tf.float64) * tf.ones(n_data, dtype=tf.float64)
    m = tf.expand_dims(m, axis=1)
    n = tf.expand_dims(n, axis=1)

    c_xy = cost_xy(x, y, scaling_coef)  # shape: [batch_size, batch_size]

    k = tf.exp(-c_xy / epsilon) + 1e-09  # add 1e-09 to prevent numerical issues
    k_t = tf.transpose(k)

    a = tf.expand_dims(tf.ones(n_data, dtype=tf.float64), axis=1)
    b = tf.expand_dims(tf.ones(n_data, dtype=tf.float64), axis=1)

    for i in range(L):
        b = m / tf.matmul(k_t, a)  # shape: [m,]
        a = n / tf.matmul(k, b)  # shape: [m,]

    return tf.reduce_sum(a * k * tf.reshape(b, (1, -1)) * c_xy)


def benchmark_loss(x, y, scaling_coef, sinkhorn_eps, sinkhorn_l, xp=None, yp=None):
    '''
    :param x: real data of shape [batch size, sequence length]
    :param y: fake data of shape [batch size, sequence length]
    :param scaling_coef: a scaling coefficient
    :param sinkhorn_eps: Sinkhorn parameter - epsilon
    :param sinkhorn_l: Sinkhorn parameter - the number of iterations
    :return: final Sinkhorn loss(and several values for monitoring the training process)
    '''
    if yp is None:
        yp = y
    if xp is None:
        xp = x
    x = tf.reshape(x, [x.shape[0], -1])
    y = tf.reshape(y, [y.shape[0], -1])
    xp = tf.reshape(xp, [xp.shape[0], -1])
    yp = tf.reshape(yp, [yp.shape[0], -1])
    loss_xy = benchmark_sinkhorn(x, y, scaling_coef, sinkhorn_eps, sinkhorn_l)
    loss_xx = benchmark_sinkhorn(x, xp, scaling_coef, sinkhorn_eps, sinkhorn_l)
    loss_yy = benchmark_sinkhorn(y, yp, scaling_coef, sinkhorn_eps, sinkhorn_l)

    loss = loss_xy - 0.5 * loss_xx - 0.5 * loss_yy

    return loss

class WGanGenerator(tf.keras.Model):
    '''
    class for WGAN generator
    Args:
        inputs, noise and confounding factor [v, z], of shape [batch size, z_dims + v_dims]
    return:
       fake samples of shape [batch size, x_dims]
    '''
    def __init__(self, n_samples, z_dims, h_dims, v_dims, x_dims, batch_size):
        super(WGanGenerator, self).__init__()
        self.n_samples = n_samples
        self.hidden_dims = h_dims
        self.batch_size = batch_size
        self.dz = z_dims
        self.dx = x_dims
        self.dv = v_dims

        self.input_dim = self.dz + self.dv
        self.input_shape1 = [self.input_dim, self.hidden_dims]
        self.input_shape2 = [self.hidden_dims, self.hidden_dims]
        self.input_shape3 = [self.hidden_dims, self.dx]

        self.w1 = self.xavier_var_creator(self.input_shape1)
        self.b1 = tf.Variable(tf.zeros(self.input_shape1[1], tf.float64))

        self.w2 = self.xavier_var_creator(self.input_shape2)
        self.b2 = tf.Variable(tf.zeros(self.input_shape2[1], tf.float64))

        self.w3 = self.xavier_var_creator(self.input_shape3)
        self.b3 = tf.Variable(tf.zeros(self.input_shape3[1], tf.float64))

    def xavier_var_creator(self, input_shape):
        xavier_stddev = 1.0 / tf.sqrt(input_shape[0] / 2.0)
        init = tf.random.normal(shape=input_shape, mean=0.0, stddev=xavier_stddev)
        init = tf.cast(init, tf.float64)
        var = tf.Variable(init, shape=tf.TensorShape(input_shape), trainable=True)
        return var

    def call(self, inputs, training=None, mask=None):
        # inputs are concatenations of z and v
        z = tf.reshape(tensor=inputs, shape=[-1, self.input_dim])
        h1 = tf.nn.relu(tf.matmul(z, self.w1) + self.b1)
        # h2 = tf.nn.relu(tf.matmul(h1, self.w2) + self.b2)
        out = tf.math.sigmoid(tf.matmul(h1, self.w3) + self.b3)
        return out


class WGanDiscriminator(tf.keras.Model):
    '''
    class for WGAN discriminator
    Args:
        inputss: real and fake samples of shape [batch size, x_dims]
    return:
       features f_x of shape [batch size, features]
    '''
    def __init__(self, n_samples, z_dims, h_dims, v_dims, batch_size):
        super(WGanDiscriminator, self).__init__()
        self.n_samples = n_samples
        self.hidden_dims = h_dims
        self.batch_size = batch_size

        self.input_dim = z_dims + v_dims
        self.input_shape1 = [self.input_dim, self.hidden_dims]
        self.input_shape2 = [self.hidden_dims, self.hidden_dims]
        self.input_shape3 = [self.hidden_dims, 1]

        self.w1 = self.xavier_var_creator(self.input_shape1)
        self.b1 = tf.Variable(tf.zeros(self.input_shape1[1], tf.float64))

        self.w2 = self.xavier_var_creator(self.input_shape2)
        self.b2 = tf.Variable(tf.zeros(self.input_shape2[1], tf.float64))

        self.w3 = self.xavier_var_creator(self.input_shape3)
        self.b3 = tf.Variable(tf.zeros(self.input_shape3[1], tf.float64))

    def xavier_var_creator(self, input_shape):
        xavier_stddev = 1.0 / tf.sqrt(input_shape[0] / 2.0)
        init = tf.random.normal(shape=input_shape, mean=0.0, stddev=xavier_stddev)
        init = tf.cast(init, tf.float64)
        var = tf.Variable(init, shape=tf.TensorShape(input_shape), trainable=True)
        return var

    def call(self, inputs, training=None, mask=None):
        # inputs are concatenations of z and v
        z = tf.reshape(tensor=inputs, shape=[self.batch_size, -1])
        z = tf.cast(z, tf.float64)
        h1 = tf.nn.relu(tf.matmul(z, self.w1) + self.b1)
        # h2 = tf.nn.sigmoid(tf.matmul(h1, self.w2) + self.b2)
        # out = tf.nn.sigmoid(tf.matmul(h1, self.w3) + self.b3)
        out = tf.matmul(h1, self.w3) + self.b3
        return out


class MINEDiscriminator(tf.keras.layers.Layer):
    '''
    class for MINE discriminator for benchmark GCIT
    '''

    def __init__(self, in_dims, output_activation='linear'):
        super(MINEDiscriminator, self).__init__()
        self.output_activation = output_activation
        self.input_dim = in_dims

        self.w1a = self.xavier_var_creator()
        self.w1b = self.xavier_var_creator()
        self.b1 = tf.Variable(tf.zeros([self.input_dim, ], tf.float64))

        self.w2a = self.xavier_var_creator()
        self.w2b = self.xavier_var_creator()
        self.b2 = tf.Variable(tf.zeros([self.input_dim, ], tf.float64))

        self.w3 = self.xavier_var_creator()
        self.b3 = tf.Variable(tf.zeros([self.input_dim, ], tf.float64))

    def xavier_var_creator(self):
        xavier_stddev = 1.0 / tf.sqrt(self.input_dim / 2.0)
        init = tf.random.normal(shape=[self.input_dim, ], mean=0.0, stddev=xavier_stddev)
        init = tf.cast(init, tf.float64)
        var = tf.Variable(init, shape=tf.TensorShape(self.input_dim, ), trainable=True)
        return var

    def mine_layer(self, x, x_hat, wa, wb, b):
        return tf.math.tanh(wa * x + wb * x_hat + b)

    def call(self, x, x_hat):
        h1 = self.mine_layer(x, x_hat, self.w1a, self.w1b, self.b1)
        h2 = self.mine_layer(x, x_hat, self.w2a, self.w2b, self.b2)
        out = self.w3 * (h1 + h2) + self.b3
        return out, tf.exp(out)


class CharacteristicFunction:
    '''
    class to construct a function that represents the characteristic function
    '''

    def __init__(self, size, x_dims, z_dims, test_size):
        self.n_samples = size
        self.hidden_dims = 20
        self.test_size = test_size

        self.input_dim = z_dims + x_dims
        self.z_dims = z_dims
        self.x_dims = x_dims
        self.input_shape1x = [self.x_dims, self.hidden_dims]
        self.input_shape1z = [self.z_dims, self.hidden_dims]
        self.input_shape1 = [self.input_dim, self.hidden_dims]
        self.input_shape2 = [self.hidden_dims, 1]

        self.w1x = self.xavier_var_creator(self.input_shape1x)
        self.b1 = tf.squeeze(self.xavier_var_creator([self.hidden_dims, 1]))

        self.w2 = self.xavier_var_creator(self.input_shape2)
        self.b2 = tf.Variable(tf.zeros(self.input_shape2[1], tf.float64))

    def xavier_var_creator(self, input_shape):
        xavier_stddev = tf.sqrt(2.0 / (input_shape[0]))
        init = tf.random.normal(shape=input_shape, mean=0.0, stddev=xavier_stddev)
        init = tf.cast(init, tf.float64)
        var = tf.Variable(init, shape=tf.TensorShape(input_shape), trainable=True)
        return var

    def update(self):
        self.w1x = self.xavier_var_creator(self.input_shape1x)
        self.b1 = tf.squeeze(self.xavier_var_creator([self.hidden_dims, 1]))
        self.w2 = self.xavier_var_creator(self.input_shape2)

    def call(self, x, z):
        # inputs are concatenations of z and v
        x = tf.reshape(tensor=x, shape=[self.test_size, -1, self.x_dims])
        z = tf.reshape(tensor=z, shape=[self.test_size, -1, self.z_dims])
        # we asssume parameter b for z to be 0
        h1 = tf.nn.sigmoid(tf.matmul(x, self.w1x) + self.b1)
        out = tf.nn.sigmoid(tf.matmul(h1, self.w2))
        return out




#
# The generate_samples_random function and rdc function were inspired by
# GCIT Github repository by Alexis Bellot1,2 Mihaela van der Schaar
# source: https://github.com/alexisbellot/GCIT
#

def generate_samples_random(size=1000, sType='CI', dx=1, dy=1, dz=20, nstd=0.05, alpha_x=0.05,
               normalize=True, seed=None, dist_z='gaussian', scaling_z = False):
    '''
    Generate CI,I or NI post-nonlinear samples
    1. Z is independent Gaussian or Laplace
    2. X = f1(<a,Z> + b + noise) and Y = f2(<c,Z> + d + noise) in case of CI
    Arguments:
        size : number of samples
        sType: CI, I, or NI
        dx: Dimension of X
        dy: Dimension of Y
        dz: Dimension of Z
        nstd: noise standard deviation
        we set f1 to be sin function and f2 to be cos function.
    Output:
        Samples X, Y, Z
    '''
    num = size

    if dist_z == 'gaussian':
        cov = np.eye(dz)
        mu = np.zeros(dz)
        Z = np.random.multivariate_normal(mu, cov, num)

    elif dist_z == 'laplace':
        Z = np.random.laplace(loc=0.0, scale=1.0, size=num*dz)
        Z = np.reshape(Z, (num, dz))

    Ax = np.random.rand(dz, dx)
    for i in range(dx):
        Ax[:, i] = Ax[:, i] / np.linalg.norm(Ax[:, i], ord=1)
    Ay = np.random.rand(dz, dy)
    for i in range(dy):
        Ay[:, i] = Ay[:, i] / np.linalg.norm(Ay[:, i], ord=1)

    Axy = np.ones((dx, dy)) * alpha_x

    if sType == 'CI':
        X = np.sin(np.matmul(Z, Ax) + nstd * np.random.multivariate_normal(np.zeros(dx), np.eye(dx), num))
        Y = np.cos(np.matmul(Z, Ay) + nstd * np.random.multivariate_normal(np.zeros(dy), np.eye(dy), num))
    elif sType == 'I':
        X = np.sin(nstd * np.random.multivariate_normal(np.zeros(dx), np.eye(dx), num))
        Y = np.cos(nstd * np.random.multivariate_normal(np.zeros(dy), np.eye(dy), num))
    else:
        X = np.sin(np.matmul(Z, Ax) + nstd * np.random.multivariate_normal(np.zeros(dx), np.eye(dx), num))
        Y = np.cos(np.matmul(X, Axy) + np.matmul(Z, Ay) + nstd * np.random.multivariate_normal(np.zeros(dx), np.eye(dx), num))

    if normalize:
        Z = (Z - Z.min()) / (Z.max() - Z.min())
        X = (X - X.min()) / (X.max() - X.min())
        Y = (Y - Y.min()) / (Y.max() - Y.min())
    if scaling_z:
        Z = Z / Z.max()
    return np.array(X), np.array(Y), np.array(Z)




def rdc(x, y, f=np.sin, k=20, s=1 / 6., n=1):
    """
    Computes the Randomized Dependence Coefficient
    x,y: numpy arrays 1-D or 2-D
         If 1-D, size (samples,)
         If 2-D, size (samples, variables)
    f:   function to use for random projection
    k:   number of random projections to use
    s:   scale parameter
    n:   number of times to compute the RDC and
         return the median (for stability)
    According to the paper, the coefficient should be relatively insensitive to
    the settings of the f, k, and s parameters.

    Source: https://github.com/garydoranjr/rdc
    """
    x = tf.reshape(x, shape=(x.shape[0], ))
    y = tf.reshape(y, shape=(y.shape[0], ))

    if n > 1:
        values = []
        for i in range(n):
            try:
                values.append(rdc(x, y, f, k, s, 1))
            except np.linalg.linalg.LinAlgError:
                pass
        return np.median(values)

    if len(x.shape) == 1: x = tf.reshape(x, shape=(-1, 1))
    if len(y.shape) == 1: y = tf.reshape(y, shape=(-1, 1))

    # Copula Transformation
    cx = np.column_stack([rankdata(xc, method='ordinal') for xc in np.transpose(x)]) / float(x.shape[0])
    cy = np.column_stack([rankdata(yc, method='ordinal') for yc in np.transpose(y)]) / float(y.shape[0])

    # Add a vector of ones so that w.x + b is just a dot product
    O = np.ones(cx.shape[0])
    X = np.column_stack([cx, O])
    Y = np.column_stack([cy, O])

    # Random linear projections
    Rx = (s / X.shape[1]) * np.random.randn(X.shape[1], k)
    Ry = (s / Y.shape[1]) * np.random.randn(Y.shape[1], k)
    X = np.dot(X, Rx)
    Y = np.dot(Y, Ry)

    # Apply non-linear function to random projections
    fX = f(X)
    fY = f(Y)

    # Compute full covariance matrix
    C = np.cov(np.hstack([fX, fY]).T)

    # Due to numerical issues, if k is too large,
    # then rank(fX) < k or rank(fY) < k, so we need
    # to find the largest k such that the eigenvalues
    # (canonical correlations) are real-valued
    k0 = k
    lb = 1
    ub = k
    while True:
        # Compute canonical correlations
        Cxx = C[:k, :k]
        Cyy = C[k0:k0 + k, k0:k0 + k]
        Cxy = C[:k, k0:k0 + k]
        Cyx = C[k0:k0 + k, :k]

        eigs = np.linalg.eigvals(np.dot(np.dot(np.linalg.pinv(Cxx), Cxy),
                                        np.dot(np.linalg.pinv(Cyy), Cyx)))

        # Binary search if k is too large
        if not (np.all(np.isreal(eigs)) and
                0 <= np.min(eigs) and
                np.max(eigs) <= 1):
            ub -= 1
            k = (ub + lb) // 2
            continue
        if lb == ub: break
        lb = k
        if ub == lb + 1:
            k = ub
        else:
            k = (ub + lb) // 2

    return np.sqrt(np.max(eigs))


def permute(x):
    n = x.shape[0]
    idx = np.random.permutation(n)
    out = x[idx]
    return out

#
# test statistics and GCIT method
# Paper link: https://arxiv.org/pdf/1907.04068.pdf
#


def mmd_squared(X, Y, gamma=1):
    X = X.reshape((len(X)), 1)
    Y = Y.reshape((len(Y)), 1)

    K_XX = rbf_kernel(X, gamma=gamma)
    K_YY = rbf_kernel(Y, gamma=gamma)
    K_XY = rbf_kernel(X, Y, gamma=gamma)

    n = K_XX.shape[0]
    m = K_YY.shape[0]

    mmd_squared = (np.sum(K_XX)-np.trace(K_XX))/(n*(n-1)) + (np.sum(K_YY)-np.trace(K_YY))/(m*(m-1)) - 2 * np.sum(K_XY) / (m * n)

    return mmd_squared


def correlation(X, Y):
    X = X.reshape((len(X)))
    Y = Y.reshape((len(Y)))
    return np.abs(np.corrcoef(X, Y)[0, 1])


def kolmogorov(X, Y):
    X = X.reshape((len(X)))
    Y = Y.reshape((len(Y)))
    return ks_2samp(X, Y)[0]


def wilcox(X, Y):
    X = X.reshape((len(X)))
    Y = Y.reshape((len(Y)))
    return wilcoxon(X, Y)[0]


def gcit_sinkhorn(n=1000, z_dim=100, simulation='type1error', statistic="rdc", batch_size=64, nstd=0.5,
                  z_dist='gaussian', n_iter=1000, current_iters=0, a_x=0.01, var_idx=1):
    # generate samples x, y, z
    # arguments: size, sType='CI', dx=1, dy=1, dz=20, nstd=1, fixed_function='linear',
    # debug=False, normalize=True, seed=None, dist_z='gaussian'
    x_dims = 1
    y_dims = 1
    if simulation == 'type1error':
        # generate samples x, y, z under null hypothesis - x and y are conditional independent
        x, y, z = generate_samples_random(size=n, sType='CI', dx=x_dims, dy=y_dims, dz=z_dim, nstd=nstd,
                                          dist_z=z_dist)

    elif simulation == 'power':
        # generate samples x, y, z under alternative hypothesis - x and y are dependent
        x, y, z = generate_samples_random(size=n, sType='dependent', dx=x_dims, dy=y_dims, dz=z_dim, nstd=nstd,
                                          dist_z=z_dist, alpha_x=a_x)

    elif simulation == 'ccle':
        x_drug, y, features = load_ccle(feature_type='mutation', normalize=False)

        ccle_selected, corrs = ccle_feature_filter(x_drug, y, threshold=0.05)

        features.index[ccle_selected]

        var_names = ['BRAF.MC_MUT', 'BRAF.V600E_MUT', 'HIP1_MUT', 'CDC42BPA_MUT', 'THBS3_MUT', 'DNMT1_MUT', 'PRKD1_MUT',
                     'FLT3_MUT', 'PIP5K1A_MUT', 'MAP3K5_MUT']
        idx = []

        for var in var_names:
            idx.append(features.T.columns.get_loc(var))

        x = x_drug[:, idx[0]]

        z = np.delete(x_drug, (idx[0]), axis=1).astype(np.float64)
        z_dim = z.shape[1]

        x = np.expand_dims(x, axis=1).astype(np.float64)
        y = np.expand_dims(y, axis=1)
        n = y.shape[0]

    elif simulation == 'brain':
        path = './data/ADNI-Mediation-new.csv'
        df = pd.read_csv(path, header=None)
        y = df.loc[:, 7].values
        age = df.loc[:, 5].values
        tr_measures = df.loc[:, 12:79].values
        ct_measures = df.loc[:, 80:].values
        all_data = np.concatenate((np.expand_dims(age, axis=1), tr_measures), axis=1)
        all_data = np.concatenate((all_data, ct_measures), axis=1)
        x_idx = np.argwhere(np.isnan(all_data))[:, 0]
        y_idx = np.argwhere(np.isnan(y))[:, 0]
        idx = np.concatenate([x_idx, y_idx])
        idx = np.unique(idx)
        idx_diff = np.arange(0, idx.shape[0])
        remove_idx = idx - idx_diff
        for i in remove_idx:
            all_data = np.delete(all_data, i, axis=0)
            y = np.delete(y, i, axis=0)

        all_data = np.delete(all_data, 0, axis=0)
        y = np.delete(y, 0, axis=0)
        x = all_data[:, var_idx]
        z = np.delete(all_data, (var_idx), axis=1).astype(np.float64)
        z_dim = z.shape[1]

        z = (z - z.min()) / (z.max() - z.min())
        x = (x - x.min()) / (x.max() - x.min())
        y = (y - y.min()) / (y.max() - y.min())

        x = np.expand_dims(x, axis=1).astype(np.float64)
        y = np.expand_dims(y, axis=1)
        n = y.shape[0]
    else:
        raise ValueError('Test does not exist.')

    # define training and testing subsets, training for learning the sampler and
    # testing for computing test statistic. Set 2/3 and 1/3 as default
    x_train, y_train, z_train = x[:int(2 * n / 3), ], y[:int(2 * n / 3), ], z[:int(2 * n / 3), ]
    # build data pipline for training set
    training_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train, z_train))
    # Repeat n epochs
    epochs = int(n_iter)
    training_dataset = training_dataset.repeat(epochs)
    batched_training_set = training_dataset.shuffle(1000).batch(batch_size*2)
    # build data pipline for test set
    x_test, y_test, z_test = x[int(2 * n / 3):, ], y[int(2 * n / 3):, ], z[int(2 * n / 3):, ]

    # no. of random and hidden dimensions
    if z_dim <= 20:
        v_dims = int(3)
        h_dims = int(3)

    else:
        v_dims = 50
        h_dims = 512

    # v_mean = tf.zeros(v_dim, dtype=tf.float64)
    v_dist = tfp.distributions.Normal(0, scale=tf.sqrt(1.0/3.0))

    # create instance of G & D
    lr = 0.0005
    scaling_coef = 1.0
    sinkhorn_eps = 0.8
    sinkhorn_l = 30
    generator = WGanGenerator(n, z_dim, h_dims, v_dims, x_dims, batch_size)
    discriminator = WGanDiscriminator(n, z_dim, h_dims, x_dims, batch_size)

    gen_clipping_val = 0.5
    gen_clipping_norm = 1.0
    w_clipping_val = 0.5
    w_clipping_norm = 1.0

    gen_optimiser = tf.keras.optimizers.Adam(lr, beta_1=0.5, clipnorm=gen_clipping_norm, clipvalue=gen_clipping_val)
    disc_optimiser = tf.keras.optimizers.Adam(lr, beta_1=0.5, clipnorm=w_clipping_norm, clipvalue=w_clipping_val)

    stat = None

    # 2. Choice of statistic
    if statistic == "corr":
        stat = correlation
    if statistic == "mmd":
        stat = mmd_squared
    if statistic == "kolmogorov":
        stat = kolmogorov
    if statistic == "wilcox":
        stat = wilcox
    if statistic == "rdc":
        stat = rdc

    @tf.function
    def update_d(real_x, real_x_p, real_z, real_z_p, v, v_p):
        gen_inputs = tf.concat([real_z, v], axis=1)
        gen_inputs_p = tf.concat([real_z_p, v_p], axis=1)
        # concatenate real inputs for WGAN discriminator (x, z)
        d_real = tf.concat([real_x, real_z], axis=1)
        d_real_p = tf.concat([real_x_p, real_z_p], axis=1)
        fake_x = generator.call(gen_inputs)
        fake_x_p = generator.call(gen_inputs_p)
        d_fake = tf.concat([fake_x, real_z], axis=1)
        d_fake_p = tf.concat([fake_x_p, real_z_p], axis=1)

        with tf.GradientTape() as disc_tape:
            f_real = discriminator.call(d_real)
            f_fake = discriminator.call(d_fake)
            f_real_p = discriminator.call(d_real_p)
            f_fake_p = discriminator.call(d_fake_p)
            # call compute loss using @tf.function + autograph

            loss1 = benchmark_loss(f_real, f_fake, scaling_coef, sinkhorn_eps, sinkhorn_l, f_real_p, f_fake_p)
            disc_loss = - loss1
        # update discriminator parameters
        d_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        disc_optimiser.apply_gradients(zip(d_grads, discriminator.trainable_variables))

    @tf.function
    def update_g(real_x, real_x_p, real_z, real_z_p, v, v_p):
        gen_inputs = tf.concat([real_z, v], axis=1)
        gen_inputs_p = tf.concat([real_z_p, v_p], axis=1)
        # concatenate real inputs for WGAN discriminator (x, z)
        d_real = tf.concat([real_x, real_z], axis=1)
        d_real_p = tf.concat([real_x_p, real_z_p], axis=1)
        with tf.GradientTape() as gen_tape:
            fake_x = generator.call(gen_inputs)
            fake_x_p = generator.call(gen_inputs_p)
            d_fake = tf.concat([fake_x, real_z], axis=1)
            d_fake_p = tf.concat([fake_x_p, real_z_p], axis=1)
            f_real = discriminator.call(d_real)
            f_fake = discriminator.call(d_fake)
            f_real_p = discriminator.call(d_real_p)
            f_fake_p = discriminator.call(d_fake_p)
            # call compute loss using @tf.function + autograph
            gen_loss = benchmark_loss(f_real, f_fake, scaling_coef, sinkhorn_eps, sinkhorn_l, f_real_p, f_fake_p)
        # update generator parameters
        generator_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gen_optimiser.apply_gradients(zip(generator_grads, generator.trainable_variables))
        return gen_loss

    for x_batch, y_batch, z_batch in batched_training_set.take(n_iter):
        # for x_batch2, y_batch2, z_batch2 in batched_training_set1.take(1):
        if x_batch.shape[0] != batch_size * 2:
            continue
        x_batch1 = x_batch[0:batch_size, ...]
        x_batch2 = x_batch[batch_size:, ...]
        z_batch1 = z_batch[0:batch_size, ...]
        z_batch2 = z_batch[batch_size:, ...]

        noise_v = v_dist.sample([batch_size, v_dims])
        noise_v = tf.cast(noise_v, tf.float64)
        noise_v_p = v_dist.sample([batch_size, v_dims])
        noise_v_p = tf.cast(noise_v_p, tf.float64)
        update_d(x_batch1, x_batch2, z_batch1, z_batch2, noise_v, noise_v_p)
        loss = update_g(x_batch1, x_batch2, z_batch1, z_batch2, noise_v, noise_v_p)

        if tf.abs(loss) < 0.1:
            break
        current_iters += 1

    test_samples = 1000
    rho = []
    test_size = z_test.shape[0]

    for i in range(test_samples):
        v = v_dist.sample([test_size, v_dims])
        v = tf.cast(v, tf.float64)
        g_inputs = tf.concat([z_test, v], axis=1)
        # generator samples from G and evaluate from D
        fake_data = generator.call(g_inputs, training=False)
        rho.append(stat(fake_data, y_test))
    tf.keras.backend.clear_session()
    rho = tf.stack(rho)
    stat_real = stat(x_test, y_test)
    # p-value computation as a two-sided test
    p_value = min(tf.reduce_sum(tf.cast(rho < stat_real, tf.float32)) / test_samples,
                  tf.reduce_sum(tf.cast(rho > stat_real, tf.float32)) / test_samples)
    return p_value


def run_experiment(params):

    model = params['model']
    sample_size = params['sample_size']
    batch_size = params['batch_size']
    z_dim = params['z_dim']
    dx = params['dx']
    dy = params['dy']
    test = params['test']
    n_test = params['n_test']
    n_iters = params['n_iters']
    eps_std = params['eps_std']
    dist_z = params['dist_z']
    alpha_x = params['alpha_x']
    alpha = params['alpha']
    alpha1 = params['alpha1']
    set_seed = params['set_seed']



    tf.random.set_seed(set_seed)
    np.random.seed(set_seed)

    if test == 'type1error':
        p_values = np.array([])
        p_values1 = []
        p_values5 = []
        test_count = 0
        for n in tqdm(range(n_test)):
            start_time = datetime.now()
            p_value = 0.0
            p_value1 = 0.0
            p_value5 = 0.0

            if model == 'gcit':
                p_value = gcit_sinkhorn(n=sample_size, z_dim=z_dim, simulation=test, statistic="rdc",
                                              batch_size=batch_size, n_iter=n_iters, nstd=eps_std, z_dist=dist_z)
            else:
                raise ValueError('Test does not exist.')

            test_count += 1
            # print("--- The %d'th iteration take %s seconds ---" % (test_count, (datetime.now() - start_time)))

            if model == 'gcit':
                p_values = np.append(p_values, p_value)
                fp = [pval < alpha / 2.0 for pval in p_values]## need to devide by 2
                final_result = np.mean(fp)
                fp1 = [pval < alpha1 / 2.0 for pval in p_values]## need to devide by 2
                final_result1 = np.mean(fp1)
            else:
                p_values = np.append(p_values, p_value)
                fp = [pval < alpha  for pval in p_values]##no need to devide by 2
                final_result = np.mean(fp)
                fp1 = [pval < alpha1 for pval in p_values]##no need to devide by 2
                final_result1 = np.mean(fp1)


            # print('The p-value is {}'.format(p_value))
            # print('Type 1 error: {} for z dimension {} with significance level {}'.format(final_result, z_dim, alpha))
            # print('Type 1 error: {} for z dimension {} with significance level {}'.format(final_result1, z_dim, alpha1))
            gc.collect()

    if test == 'power':
        p_values = np.array([])
        p_values1 = []
        p_values5 = []
        test_count = 0
        for n in tqdm(range(n_test)):
            start_time = datetime.now()
            p_value = 0.0
            p_value1 = 0.0
            p_value5 = 0.0

            if model == 'gcit':
                p_value = gcit_sinkhorn(n=sample_size, z_dim=z_dim, simulation=test, statistic="rdc",
                                              batch_size=batch_size, n_iter=n_iters,
                                              nstd=eps_std, z_dist=dist_z)
            else:
                raise ValueError('Test does not exist.')

            test_count += 1
            # print("--- The %d'th iteration take %s seconds ---" % (test_count, (datetime.now() - start_time)))

            if model == 'gcit':
                p_values = np.append(p_values, p_value)
                fp = [pval < alpha / 2.0 for pval in p_values]## need to devide by 2
                final_result = np.mean(fp)
                fp1 = [pval < alpha1 / 2.0 for pval in p_values]## need to devide by 2
                final_result1 = np.mean(fp1)
            else:
                p_values = np.append(p_values, p_value)
                fp = [pval < alpha  for pval in p_values]
                final_result = np.mean(fp)
                fp1 = [pval < alpha1  for pval in p_values]
                final_result1 = np.mean(fp1)
            # print('The p-value is {}'.format(p_value))
            # print('Power: {} for z dimension {} with significance level {}'.format(final_result,z_dim, alpha))
            # print('Power: {} for z dimension {} with significance level {}'.format(final_result1,z_dim, alpha1))
            gc.collect()

    print('Emp Rej Rate: {} for z dimension {} with significance level {}'.format(final_result, z_dim, alpha))
    print('Emp Rej Rate: {} for z dimension {} with significance level {}'.format(final_result1, z_dim, alpha1))

    return p_values


In [ ]:
# @title code to get Emperical Rejection Rate for Figure 2 (a) GCIT

param["test"] = "type1error"

for z_dim in [50, 100, 150, 200, 250]:
    param["z_dim"] = z_dim
    p_val_list = run_experiment(param)

In [ ]:
# @title code to get Size Adjusted Power for Figure 2 (b) GCIT

param["test"] = "power"
param["z_dim"] = 200

param["alpha"] = 0.038 # 5% quantile of the p_val_list from previous block when param["test"] = "type1error" param["z_dim"] = 200
param["alpha1"] = 0.078 # 10% quantile of the p_val_list from previous block when param["test"] = "type1error" param["z_dim"] = 200


for alpha_x in [0.15, 0.30, 0.45, 0.60, 0.75]:
    param["alpha_x"] = alpha_x
    p_val_list = run_experiment(param)